In [ ]:
# load data

In [1]:
import geopandas as gpd


# PPR data with NDVI calculated
park_df = gpd.read_file('output_files/PPRwithNDVI.geojson')
park_df = park_df.to_crs("EPSG:4326")

In [2]:
park_df.head()

,OBJECTID,PUBLIC_NAME,PARENT_NAME,NESTED,OFFICIAL_NAME,LABEL,ALIAS,DPP_ASSET_ID,ADDRESS_911,ZIP_CODE,...,COUNCIL_DISTRICT,POLICE_DISTRICT,CITY_SCALE_MAPS,LOCAL_SCALE_MAPS,PROGRAM_SITES,COMMENTS,Shape__Area,Shape__Length,median_NDVI,geometry
0,1,William H (Billy) Markward Memorial Playground,Schuylkill River Park,Y,William H. (Billy) Markward Memorial Playground,Markward,Joseph O'Connor Memorial Pool,1903,400 S TANEY ST,19146,...,2,9;17,Y,Y,Y,Markward site includes the O'Connor Pool,25871.441406,1094.402639,0.317718,"MULTIPOLYGON (((-75.18307 39.94826, -75.18314 ..."
1,2,Schuylkill Banks,Schuylkill Banks,N,NaN,Schuylkill Banks,NaN,2794,2425 CHESTNUT ST,19103,...,2;5,9;17,Y,Y,N,Bill 160113 (4/14/16) -South of South St Bridge,101477.171875,9300.670275,0.222291,"POLYGON ((-75.18103 39.96361, -75.18075 39.963..."
2,3,Schuylkill River Park,Schuylkill River Park,N,NaN,Schuylkill River Park,NaN,2992,400 S TANEY ST,19146,...,2,9,Y,Y,N,NaN,27216.929688,799.048584,0.326216,"POLYGON ((-75.18314 39.94821, -75.18307 39.948..."
3,4,Frankford Boat Launch,Frankford Boat Launch,N,Frankford Boat Launch,Frankford Launch,PA Fish & Boat;Frankford Arsenal Boat Launch,3090,5625 TACONY ST,19137,...,6,15,Y,Y,N,NaN,78515.335938,1585.376114,0.228746,"POLYGON ((-75.05596 40.00750, -75.05601 40.007..."
4,5,Robert A Borski Park,Robert A Borski Park,N,Robert A. Borski Park,Borski,Orthodox Street Parcel; Bridesburg Riverfront ...,3091,3101 ORTHODOX ST,19137,...,6,15,Y,Y,N,Officially acquired from PAID in 10/2019. Doc ...,72926.187500,1304.901416,0.222559,"POLYGON ((-75.06707 39.99007, -75.06713 39.990..."


In [3]:
ppr_adult = gpd.read_file('data/PPR_Adult_Exercise_Equipment.geojson')

In [4]:
park_df = park_df.to_crs("EPSG:4326")
ppr_adult = ppr_adult.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_adult, how="left", op="intersects")

park_df['Adult_Exercise_Equipment'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['PARK_NAME'].isnull().all() else 0)


D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [5]:
ppr_art = gpd.read_file('data/PPR_Art_Monuments_point.geojson')

In [6]:
ppr_art = ppr_art.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_art, how="left", op="intersects")

park_df['Art_Monuments'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['GENERALLOC'].isnull().all() else 0)

D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [7]:
ppr_boat = gpd.read_file('data/PPR_Boat_Launches.geojson')

In [8]:
ppr_boat.head()
#ParkName: 'PARENT_NAME'

,OBJECTID,AMENITY_NAME,PARENT_NAME,ALIAS,RAMP,DOCK,PARKING,COMMENTS,geometry
0,1,Linden Avenue Boat Ramp,Pleasant Hill Park,The River at Linden Ave,Y,N,Y,Unlimited Horsepower; Closed until further not...,POINT (-8348072.336 4871909.797)
1,2,East Park Canoe House Launch,East Fairmount Park,Temple Boathouse Dock,N,Y,Y,"Mainly for Kayaks & Canoes, several locations ...",POINT (-8370545.704 4865035.602)
2,3,Walnut Street Dock,Schuylkill Banks,,N,Y,N,This boat launch is only designed to accommoda...,POINT (-8369115.409 4858947.112)
3,4,Bartrams Garden Dock,Bartrams Garden,Bartram's Garden Community Boathouse and Dock,N,Y,Y,Pedestrian and kayaks,POINT (-8372136.322 4856267.822)
4,5,Tugboat House Boat Launch,West Fairmount Park,Tugboat House at Belmont Landing;Phila Rowing ...,N,Y,Y,"Private, Historic. It is only designed to acco...",POINT (-8371683.391 4863123.886)


In [9]:
park_names_set = set(ppr_boat['PARENT_NAME'])

# Now, create a new column in 'park_df' based on whether 'PARENT_NAME' is in the set of park names
park_df['Boat_Launches'] = park_df['PARENT_NAME'].isin(park_names_set).astype(int)


In [10]:
ppr_water = gpd.read_file('data/PPR_Hydration_Stations.geojson')

In [11]:
ppr_water.head()

,OBJECTID,AMENITY_NAME,PARK_NAME,DISPLAY_LABEL,STATION_TYPE,LOCATION,DISPLAY_TYPE,COMMENTS,DATA_SOURCE,geometry
0,1,Verree Road Picnic Area Restrooms,Pennypack Park,,FOUNTAIN,OUTDOOR,ALL,,Nearmap 2023,POINT (-8355824.445 4878643.962)
1,2,Lawncrest Recreation Center,Lawncrest Recreation Center,,FOUNTAIN,INDOOR,ALL,,Initial Trails Web App,POINT (-8360216.459 4872555.886)
2,3,Franklin Delano Roosevelt Recreation Center,Franklin Delano Roosevelt Playground,Roosevelt,HYDRATION,INDOOR,ALL,Installed 4/2019,Hydrate Philly Project,POINT (-8354926.438 4869759.049)
3,4,Feltonville Recreation Center,Feltonville Recreation Center,Feltonville,HYDRATION,INDOOR,ALL,,Hydrate Philly Project,POINT (-8362447.086 4869056.147)
4,5,Samuel F Houston Recreation Center,Samuel F Houston Playground,,FOUNTAIN,OUTDOOR,ALL,,Nearmap 2023,POINT (-8374705.927 4876032.183)


In [12]:
ppr_water = ppr_water.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_water, how="left", op="intersects")

park_df['Hydration_Stations'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['PARK_NAME'].isnull().all() else 0)

D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [13]:
ppr_picnic = gpd.read_file('data/PPR_Picnic_Sites.geojson')
ppr_picnic.head()

,OBJECTID,PARK_NAME,SITE_NUMBER,TABLE_COUNT,PERMITTABLE,COMMENTS,DATA_SOURCE,geometry
0,1,Belmont Grove,1.0,10,Y,West Fairmount Park- Belmont Grove Picnic Pavi...,Nearmap 2023,POINT (-8372896.082 4864857.710)
1,2,Franklin Delano Roosevelt Park,7.0,3,Y,,Nearmap 2023,POINT (-8369351.632 4851922.888)
2,3,Franklin Delano Roosevelt Park,8.0,3,Y,,Nearmap 2023,POINT (-8369402.211 4851749.257)
3,4,Franklin Delano Roosevelt Park,9.0,3,Y,,Nearmap 2023,POINT (-8369401.122 4851695.715)
4,5,Franklin Delano Roosevelt Park,10.0,3,Y,,Nearmap 2023,POINT (-8369294.204 4851605.482)


In [15]:
ppr_picnic = ppr_picnic.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_picnic, how="left", op="intersects")

park_df['Picnic_Sites'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['PARK_NAME'].isnull().all() else 0)

D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [16]:
ppr_plg = gpd.read_file('data/PPR_Playground.geojson')
ppr_plg.head()

,OBJECTID,ADDRESS,AGE_INT,SITE,RESTROOMS,WEBSITE,geometry
0,1,4131 UNRUH AVE,2-12,Vogt Recreation Center,no,http://www.phila.gov/parksandrecreation/findaf...,POINT (2724577.417 264704.947)
1,2,5700 TORRESDALE AVE,2-12,Moss Playground,yes,http://www.phila.gov/parksandrecreation/findaf...,POINT (2721053.445 260156.206)
2,3,4740 WOODLAND AVE,5-12,48th & Woodland Playground,no,http://www.phila.gov/parksandrecreation/findaf...,POINT (2680794.442 231863.978)
3,4,1900 WAKELING ST,2-12,Gambrel,no,http://www.phila.gov/parksandrecreation/findaf...,POINT (2717228.664 259379.700)
4,5,400-32 E INDIANA AVE,2-12,Hissey Playground,no,http://www.phila.gov/parksandrecreation/findaf...,POINT (2704337.726 251797.842)


In [17]:
ppr_plg = ppr_plg.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_plg, how="left", op="intersects")

park_df['Playground'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['SITE'].isnull().all() else 0)

D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [18]:
ppr_pls = gpd.read_file('data/PPR_Playstreets_Locations.geojson')
ppr_pls.head()

,OBJECTID,BLOCK_NAME,ZIP_CODE,YEAR,COMMENTS,DATA_SOURCE,geometry
0,1,2900 Block of N CAMAC ST,19133,2017,NaN,2017 Playstreets Program,POINT (-8365746.638 4865556.306)
1,2,2900 Block of N BAILEY ST,19132,2017,NaN,2017 Playstreets Program,POINT (-8368293.516 4865992.715)
2,3,2200 Block of N FRANKLIN ST,19133,2017,NaN,2017 Playstreets Program,POINT (-8365209.592 4863876.458)
3,4,2000 Block of N LAWRENCE ST,19122,2017,NaN,2017 Playstreets Program,POINT (-8364707.919 4863324.417)
4,5,2200 Block of N BANCROFT ST,19132,2017,NaN,2017 Playstreets Program,POINT (-8366756.668 4864137.501)


In [19]:
ppr_pls = ppr_pls[ppr_pls['YEAR'] == '2023']

In [20]:
ppr_pls = ppr_pls.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_pls, how="left", op="intersects")

park_df['Playstreets_Locations'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['BLOCK_NAME'].isnull().all() else 0)

D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [21]:
ppr_poolNspray = gpd.read_file('data/PPR_pools_spraygrounds.geojson')
ppr_poolNspray.head()

,OBJECTID,ZIPCODE,ADDRESS,FACILITY,LINK,FEATURE,HOURS,geometry
0,1,19141.0,851 W Olney Ave,Cherashore Playground,http://www.phila.gov/parksandrecreation/findaf...,Pool,"Monday through Friday 11:00am-7:00pm, Saturday...",POINT (2700217.251 267530.407)
1,2,19120.0,200-64 E Olney Ave,Ziehler Playground,http://www.phila.gov/parksandrecreation/findaf...,Pool,"Monday through Friday 11:00am-7:00pm, Saturday...",POINT (2705177.749 266470.594)
2,3,19128.0,201 Fountain St,Hillside Recration Center,http://www.phila.gov/parksandrecreation/findaf...,Pool,"Monday through Friday 11:00am-7:00pm, Saturday...",POINT (2674207.999 265125.905)
3,4,19135.0,4131 Unruh Ave,Vogt Recreation Center,http://www.phila.gov/parksandrecreation/findaf...,Pool,"Monday through Friday 11:00am-7:00pm, Saturday...",POINT (2724680.249 265017.749)
4,5,19128.0,5822-24 Ridge Ave,Kendrick Recreation Center,http://www.phila.gov/parksandrecreation/findaf...,Pool,"Monday through Friday 11:00am-7:00pm, Saturday...",POINT (2679362.251 263802.218)


In [22]:
ppr_poolNspray = ppr_poolNspray.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_poolNspray, how="left", op="intersects")

park_df['Pools_Sprayground'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['FACILITY'].isnull().all() else 0)

D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [23]:
ppr_tennis = gpd.read_file('data/PPR_Tennis_Courts.geojson')
ppr_tennis.head()

,OBJECTID,PARK_NAME,COURT_NUMBER,COMMENTS,DATA_SOURCE,geometry
0,1,Franklin Delano Roosevelt Park,5,,Nearmap 2023,POINT (-8368581.053 4852057.848)
1,2,Hunting Park,1,,Nearmap 2023,POINT (-8364757.378 4868147.943)
2,3,Fisher Park,2,,Nearmap 2023,POINT (-8363577.723 4872304.467)
3,4,Eastwick Regional Playground,3,Renovated in 2021 (share pickleball),Nearmap 2023,POINT (-8376855.993 4852113.974)
4,5,Wisters Woods Park,0,Operated by Woodford Tennis Courts,Property Team 2024,POINT (-8366578.646 4870962.634)


In [24]:
ppr_tennis = ppr_tennis.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_tennis, how="left", op="intersects")

park_df['Tennis_Court'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['PARK_NAME'].isnull().all() else 0)

D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [25]:
ppr_arg = gpd.read_file('data/PPR_Urban_Agriculture_Projects.geojson')
ppr_arg.head()

,OBJECTID,PROJECT_NAME,PARK_NAME,ADDRESS,CITY,STATE,ZIP_CODE,CONTACT_EMAIL,CONTACT_WEBSITE,PUBLIC_HOURS,PROJECT_ALIAS,SITE_DESCRIPTION,PROJECT_STATUS,PROGRAM,COUNCIL_DISTRICT,COMMENTS,geometry
0,1,Fox Chase Farm,Fox Chase Farm,8500 Pine Rd,Philadelphia,PA,19111,info@foxchasefarm.org,https://www.foxchasefarm.org/,Only open to the public for specific events.,,Located at 8500 Pine Rd. and Pennypack Creek.,ACTIVE,FPHPT Lease,10,Located north of Pennypack Park. The farm is a...,POINT (-8357261.294 4879130.797)
1,2,Master Gardeners Demonstration Vegetable Garde...,West Fairmount Park,100 N Horticultural Dr,Philadelphia,PA,19131,jes6956@psu.edu,https://extension.psu.edu/programs/master-gard...,8am–6pm (April 1-October 31); \n8am–5pm (Novem...,,Located on the south side of the Fairmount Hor...,ACTIVE,N/A,4,NaN,POINT (-8372412.101 4863439.428)
2,3,Greenland Nursery,West Fairmount Park,3852 W Ford Rd,Philadelphia,PA,19131,farmphilly@phila.gov,N/A,N/A,,Located in West Fairmount Park next to the Org...,ACTIVE,N/A,4,NaN,POINT (-8371561.912 4865297.458)
3,4,Saul High School - Henry Got Crops,Wissahickon Valley Park,7095 Henry Ave,Philadelphia,PA,19128,https://saul.philasd.org/contact-us/,https://saul.philasd.org/,Seasonal farmers market: Tuesdays 10 am - 7 pm...,Saul High School Farm,Located across Henry Ave from Saul High School.,ACTIVE,School District of Philadelphia,4,NaN,POINT (-8373373.046 4873123.862)
4,5,Sankofa Community Farm,Bartrams Garden,5400 Lindbergh Blvd,Philadelphia,PA,19143,info@bartramsgarden.org,https://www.bartramsgarden.org/farm/,Farmstand (June-November) on Thursday (3:30pm-...,Sankofa Community Farm at Bartrams Garden,Located at the southern end of Bartrams Garden...,ACTIVE,The John Bartram Association,3,There is an orchard and other urban agricultur...,POINT (-8372720.028 4855847.303)


In [26]:
ppr_arg = ppr_arg.to_crs("EPSG:4326")

joined_df = gpd.sjoin(park_df, ppr_arg, how="left", op="intersects")

park_df['Urban_Agriculture'] = joined_df.groupby(joined_df.index).apply(lambda x: 1 if not x['PARK_NAME'].isnull().all() else 0)

D:\Anaconda\Install\envs\musa-550-fall-2023\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [27]:
park_df.head()

,OBJECTID,PUBLIC_NAME,PARENT_NAME,NESTED,OFFICIAL_NAME,LABEL,ALIAS,DPP_ASSET_ID,ADDRESS_911,ZIP_CODE,...,Adult_Exercise_Equipment,Art_Monuments,Boat_Launches,Hydration_Stations,Picnic_Sites,Playground,Playstreets_Locations,Pools_Sprayground,Tennis_Court,Urban_Agriculture
0,1,William H (Billy) Markward Memorial Playground,Schuylkill River Park,Y,William H. (Billy) Markward Memorial Playground,Markward,Joseph O'Connor Memorial Pool,1903,400 S TANEY ST,19146,...,0,0,0,1,1,1,0,1,1,0
1,2,Schuylkill Banks,Schuylkill Banks,N,NaN,Schuylkill Banks,NaN,2794,2425 CHESTNUT ST,19103,...,0,0,1,1,0,0,0,0,0,0
2,3,Schuylkill River Park,Schuylkill River Park,N,NaN,Schuylkill River Park,NaN,2992,400 S TANEY ST,19146,...,0,0,0,0,1,0,0,0,0,0
3,4,Frankford Boat Launch,Frankford Boat Launch,N,Frankford Boat Launch,Frankford Launch,PA Fish & Boat;Frankford Arsenal Boat Launch,3090,5625 TACONY ST,19137,...,0,0,1,0,0,0,0,0,0,0
4,5,Robert A Borski Park,Robert A Borski Park,N,Robert A. Borski Park,Borski,Orthodox Street Parcel; Bridesburg Riverfront ...,3091,3101 ORTHODOX ST,19137,...,0,0,0,0,0,0,0,0,0,0


In [28]:
print(park_df.columns)

Index(['OBJECTID', 'PUBLIC_NAME', 'PARENT_NAME', 'NESTED', 'OFFICIAL_NAME',
       'LABEL', 'ALIAS', 'DPP_ASSET_ID', 'ADDRESS_911', 'ZIP_CODE',
       'ADDRESS_BRT', 'ALIAS_ADDRESS', 'ACREAGE', 'PROPERTY_CLASSIFICATION',
       'PPR_USE', 'PPR_PROG_DISTRICT', 'PPR_OPS_DISTRICT', 'COUNCIL_DISTRICT',
       'POLICE_DISTRICT', 'CITY_SCALE_MAPS', 'LOCAL_SCALE_MAPS',
       'PROGRAM_SITES', 'COMMENTS', 'Shape__Area', 'Shape__Length',
       'median_NDVI', 'geometry', 'Adult_Exercise_Equipment', 'Art_Monuments',
       'Boat_Launches', 'Hydration_Stations', 'Picnic_Sites', 'Playground',
       'Playstreets_Locations', 'Pools_Sprayground', 'Tennis_Court',
       'Urban_Agriculture'],
      dtype='object')


In [29]:
columns_to_keep = ['OBJECTID', 'PUBLIC_NAME', 'PARENT_NAME', 'Shape__Area', 'median_NDVI', 'geometry', 
                   'Adult_Exercise_Equipment', 'Art_Monuments',
       'Boat_Launches', 'Hydration_Stations', 'Picnic_Sites', 'Playground',
       'Playstreets_Locations', 'Pools_Sprayground', 'Tennis_Court',
       'Urban_Agriculture']

park_df = park_df[columns_to_keep]

In [30]:
park_df.head()

,OBJECTID,PUBLIC_NAME,PARENT_NAME,Shape__Area,median_NDVI,geometry,Adult_Exercise_Equipment,Art_Monuments,Boat_Launches,Hydration_Stations,Picnic_Sites,Playground,Playstreets_Locations,Pools_Sprayground,Tennis_Court,Urban_Agriculture
0,1,William H (Billy) Markward Memorial Playground,Schuylkill River Park,25871.441406,0.317718,"MULTIPOLYGON (((-75.18307 39.94826, -75.18314 ...",0,0,0,1,1,1,0,1,1,0
1,2,Schuylkill Banks,Schuylkill Banks,101477.171875,0.222291,"POLYGON ((-75.18103 39.96361, -75.18075 39.963...",0,0,1,1,0,0,0,0,0,0
2,3,Schuylkill River Park,Schuylkill River Park,27216.929688,0.326216,"POLYGON ((-75.18314 39.94821, -75.18307 39.948...",0,0,0,0,1,0,0,0,0,0
3,4,Frankford Boat Launch,Frankford Boat Launch,78515.335938,0.228746,"POLYGON ((-75.05596 40.00750, -75.05601 40.007...",0,0,1,0,0,0,0,0,0,0
4,5,Robert A Borski Park,Robert A Borski Park,72926.187500,0.222559,"POLYGON ((-75.06707 39.99007, -75.06713 39.990...",0,0,0,0,0,0,0,0,0,0


In [31]:
park_df.to_file("output_files/PPR_with_NDVI_Amenity.geojson", driver='GeoJSON')

In [10]:
import geopandas as gpd
park_df = gpd.read_file('output_files/PPR_with_NDVI_Amenity.geojson')

In [3]:
import pandas as pd
reviews_df = pd.read_csv('output_files/review_sent_emo.csv')

In [4]:
reviews_df.head()

,timestamp,author,user_id,review_text,rating,place_id,ParkName,Lat,Long,time,...,optimism,pride,realization,relief,remorse,sadness,surprise,weights,cluster,new_emotion
0,2023-12-30 19:08:54,Eleni Belisonzi,106030427780813104531,A neighbor jewel.,5,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,2023-12-30 19:08:54,...,0.001877,0.000380,0.005319,0.000340,0.000415,0.002123,0.001158,0.585360,7,Relaxation
1,2023-08-14 17:51:59,Marta Harrison,116743363301808220011,"visited in a nostalgia trip, as I took my todd...",4,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,2023-08-14 17:51:59,...,0.004074,0.014304,0.033353,0.027964,0.000763,0.004404,0.002918,0.450411,3,Engagement
2,2023-06-06 11:43:07,Akshita Srivastava,111921958911249462562,"Hung out in peace, great for solitude lovers",5,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,2023-06-06 11:43:07,...,0.004414,0.010218,0.006221,0.008279,0.000474,0.004261,0.000506,0.395096,0,Inspiration
3,2023-04-20 20:58:42,Andy Reinoso,116446471570828426598,Nice little park in the middle of old historic...,4,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,2023-04-20 20:58:42,...,0.009164,0.007238,0.011970,0.005709,0.000175,0.001040,0.000501,0.361360,0,Inspiration
4,2023-03-20 22:03:31,Moaaz Marzouk,110355050307790944461,Nice small place for kids,4,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,2023-03-20 22:03:31,...,0.005207,0.002069,0.005965,0.000588,0.000234,0.000922,0.000758,0.340700,0,Inspiration


In [5]:
reviews_df['time'] = reviews_df['time'].astype(str)

reviews_df_2324 = reviews_df[reviews_df['time'].str.startswith(('2023', '2024'))]

In [6]:
park_plot_new = reviews_df_2324.drop_duplicates(subset='ParkName')[['ParkName', 'Long', 'Lat']]

review_count_by_park_new = reviews_df_2324.groupby('ParkName').size().reset_index(name='ReviewCount')

# Merge the review count with the park_plot_new DataFrame
park_plot_new = park_plot_new.merge(review_count_by_park_new, on='ParkName', how='left')

park_plot_new.head()

,ParkName,Long,Lat,ReviewCount
0,Delancey Park,-75.147900,39.944080,7
1,Mifflin Square,-75.156519,39.919477,13
2,William Lanier Park,-75.193880,39.933620,11
3,Marian Winfield Hissey Recreation Center,-75.123340,39.994330,4
4,Palmer Park,-75.132540,39.974890,15


In [7]:
weighted_counts = reviews_df_2324.groupby(['ParkName', 'new_emotion']).apply(
    lambda x: (x['weights']).sum()
).unstack(fill_value=0)

# Calculate the total weights per park to normalize
total_weights_per_park_weighted = weighted_counts.sum(axis=1)

# Calculate weighted percentages
park_cluster_percentages_weighted = weighted_counts.div(total_weights_per_park_weighted, axis=0) * 100

# Rename the columns to include '_percentage'
park_cluster_percentages_weighted.columns = [f'{emotion}_percentage_weighted' for emotion in park_cluster_percentages_weighted.columns]

# Round the percentages and check the first few rows
park_emo_weighted = park_cluster_percentages_weighted.round(5)

In [8]:
park_plot_new = park_plot_new.merge(park_emo_weighted, on='ParkName', how='left')

park_plot_new.head()

park_plot_new = gpd.GeoDataFrame(
    park_plot_new, 
    geometry=gpd.points_from_xy(park_plot_new['Long'], park_plot_new['Lat'])
)

# Set the original CRS for the GeoDataFrame
park_plot_new.set_crs('EPSG:4326', inplace=True)

,ParkName,Long,Lat,ReviewCount,Discovery_percentage_weighted,Engagement_percentage_weighted,Frustration & Annoyance_percentage_weighted,Inspiration_percentage_weighted,Relaxation_percentage_weighted,Sorrow_percentage_weighted,geometry
0,Delancey Park,-75.147900,39.944080,7,0.0,16.53567,0.00000,61.97435,21.48998,0.00000,POINT (-75.14790 39.94408)
1,Mifflin Square,-75.156519,39.919477,13,0.0,21.64425,0.00000,25.40431,52.95144,0.00000,POINT (-75.15652 39.91948)
2,William Lanier Park,-75.193880,39.933620,11,0.0,35.98793,0.00000,55.36721,8.64486,0.00000,POINT (-75.19388 39.93362)
3,Marian Winfield Hissey Recreation Center,-75.123340,39.994330,4,0.0,29.11786,18.38961,27.71509,24.77745,0.00000,POINT (-75.12334 39.99433)
4,Palmer Park,-75.132540,39.974890,15,0.0,26.58572,0.00000,62.00461,11.40967,0.00000,POINT (-75.13254 39.97489)
...,...,...,...,...,...,...,...,...,...,...,...
86,Frankford and Pennypack Park,-75.035803,40.049551,25,0.0,23.77529,0.00000,52.69265,19.56206,3.97001,POINT (-75.03580 40.04955)
87,Pennypack Park,-75.035803,40.049551,25,0.0,23.77529,0.00000,52.69265,19.56206,3.97001,POINT (-75.03580 40.04955)
88,Gold Star Park,-75.155180,39.932940,7,0.0,23.30490,0.00000,76.69510,0.00000,0.00000,POINT (-75.15518 39.93294)
89,Frank Palumbo Sr Park,-75.155587,39.939260,3,0.0,0.00000,0.00000,100.00000,0.00000,0.00000,POINT (-75.15559 39.93926)


In [11]:
# Convert feet to meters
feet_to_meters = 80 / 3.281  # Convert 80 feet to meters

# Function to perform the join
def join_parks(park_plot_new, park_df):
    # Attempt to join by names
    merged_by_name = pd.merge(park_plot_new, park_df, left_on='ParkName', right_on='PARENT_NAME', how='left')
    
    # Check if any matches are found
    if merged_by_name['PARENT_NAME'].notna().any():
        return merged_by_name
    else:
        # If no matches, perform spatial join with buffer
        park_plot_new['buffered_geometry'] = park_plot_new.geometry.buffer(feet_to_meters)
        park_reg = gpd.sjoin(park_plot_new.set_geometry('buffered_geometry'), park_df, how='left', predicate='intersects')
        return park_reg

# Apply the join function
park_reg = join_parks(park_plot_new, park_df)

# Display the result
park_reg.head()

,ParkName,Long,Lat,ReviewCount,Discovery_percentage_weighted,Engagement_percentage_weighted,Frustration & Annoyance_percentage_weighted,Inspiration_percentage_weighted,Relaxation_percentage_weighted,Sorrow_percentage_weighted,...,Art_Monuments,Boat_Launches,Hydration_Stations,Picnic_Sites,Playground,Playstreets_Locations,Pools_Sprayground,Tennis_Court,Urban_Agriculture,geometry_y
0,Delancey Park,-75.147900,39.944080,7,0.0,16.53567,0.00000,61.97435,21.48998,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,"POLYGON ((-75.14777 39.94436, -75.14777 39.944..."
1,Mifflin Square,-75.156519,39.919477,13,0.0,21.64425,0.00000,25.40431,52.95144,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,"POLYGON ((-75.15591 39.91882, -75.15592 39.918..."
2,William Lanier Park,-75.193880,39.933620,11,0.0,35.98793,0.00000,55.36721,8.64486,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-75.19315 39.93246, -75.19317 39.932..."
3,Marian Winfield Hissey Recreation Center,-75.123340,39.994330,4,0.0,29.11786,18.38961,27.71509,24.77745,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"POLYGON ((-75.12306 39.99454, -75.12300 39.994..."
4,Palmer Park,-75.132540,39.974890,15,0.0,26.58572,0.00000,62.00461,11.40967,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-75.13271 39.97453, -75.13272 39.974..."


In [51]:
park_reg.head(93)

,ParkName,Long,Lat,ReviewCount,Discovery_percentage_weighted,Engagement_percentage_weighted,Frustration & Annoyance_percentage_weighted,Inspiration_percentage_weighted,Relaxation_percentage_weighted,Sorrow_percentage_weighted,...,Adult_Exercise_Equipment,Art_Monuments,Boat_Launches,Hydration_Stations,Picnic_Sites,Playground,Playstreets_Locations,Pools_Sprayground,Tennis_Court,Urban_Agriculture
0,Delancey Park,-75.147900,39.944080,7,10.80638,0.00000,0.00000,21.48998,67.70364,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,Mifflin Square,-75.156519,39.919477,13,0.00000,0.00000,0.00000,52.95144,47.04856,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,William Lanier Park,-75.193880,39.933620,11,8.38614,0.00000,0.00000,8.64486,82.96899,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Marian Winfield Hissey Recreation Center,-75.123340,39.994330,4,0.00000,0.00000,0.00000,43.16705,56.83295,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Palmer Park,-75.132540,39.974890,15,12.61043,0.00000,7.12820,11.40821,68.85316,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Wissahickon Valley Park,-75.217331,40.056211,327,9.87345,0.00000,1.40734,16.01118,72.70803,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,Wissahickon Valley Park,-75.217331,40.056211,327,9.87345,0.00000,1.40734,16.01118,72.70803,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,Franklin Delano Roosevelt Park,-75.183810,39.902140,266,10.76470,0.00000,0.88556,26.93912,61.41062,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
91,Franklin Delano Roosevelt Park,-75.183810,39.902140,266,10.76470,0.00000,0.88556,26.93912,61.41062,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
park_reg.to_csv('output_files/PrepforRegression.csv', index=False)